In [57]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from wordcloud import WordCloud
from datetime import datetime

In [58]:
twitter = Twitter()
okt = Okt()

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [134]:
def data_cleansing_character(d):                      # 정규표현식을 활용 but, 목차 제거 등은 다른 방법이 필요한 듯
    parse = re.sub('[-=+,#/\?:^$.@*\"※~⚫&%ㆍ!』\"\'\\‘|\(\)\[\]\<\>`\'…》]'," ",d)          #parse = re.sub('[^\w\s]',"",d)   이것도 되나?
    return parse

In [135]:
def data_ckonlpy(twitter, d):
    konlpy_array = twitter.morphs(d)
    return konlpy_array

In [136]:
def data_cleansing(twitter, name, data_word):
    
    data_cleansing_file = open(name,'r',-1,'utf-8')           # -1 은 버퍼 / utf-8 추가 or 파일 인코딩 ANSI 변경으로 인코딩 문제 해결 가능
    data = data_cleansing_file.read()

    c = data_cleansing_character(data)

    data_word = data_ckonlpy(twitter, c)


In [137]:
data_list = []
for i in data_word:
    data_list += i

In [138]:
#형태소 분석 결과 파일 저장
def prepare_data_set(data_list, data_set):
    cnt = 0
    for word in data_list:
        if len(word) <= 1:            # 단어 길이가 1 이면 --> 무의미하다고 가정
            continue
        if word == " ":  # 단어 길이가 1 이면 --> 무의미하다고 가정
            continue
        data_set.append(word)
        cnt += 1

In [139]:
data_set = []
prepare_data_set(data_list, data_set)

In [140]:
def all_data_set(twitter, data):
    
    for i in range(1,5):
        name = str(i)+'.txt'
        
        data_cleansing(twitter, name, data_word)
    
        data_list = []
        data_set = []
        
        for i in data_word:
            data_list += i
                
        prepare_data_set(data_list, data_set)
        data.append(data_set)
        

In [144]:
data = []

all_data_set(twitter, data)

In [155]:
len(data)

4

In [157]:
dictionary = corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(0, 1), (1, 6), (2, 1), (3, 2), (4, 14), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 7), (20, 1), (21, 2), (22, 1), (23, 3), (24, 1), (25, 1), (26, 3), (27, 2), (28, 5), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 4), (38, 1), (39, 9), (40, 2), (41, 14), (42, 5), (43, 1), (44, 2), (45, 7), (46, 1), (47, 2), (48, 16), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 4), (68, 78), (69, 8), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 6), (77, 5), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 4), (84, 1), (85, 1), (86, 1), (87, 1), (88, 3), (89, 2), (90, 1), (91, 5), (92, 1), (93, 8), (94, 4), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 3), (103, 4), (104, 2), (105, 1), (106, 1), (107, 2), (108, 1), (109, 4), (110,

In [174]:
NUM_TOPICS = 2 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=100)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.002*"기술" + 0.002*"배터리" + 0.002*"유출" + 0.002*"화학" + 0.002*"소송" + 0.002*"보안" + 0.002*"산업" + 0.002*"보호" + 0.002*"미국" + 0.002*"핵심"')
(1, '0.055*"기술" + 0.027*"화학" + 0.021*"배터리" + 0.021*"유출" + 0.020*"보안" + 0.018*"소송" + 0.017*"산업" + 0.016*"보호" + 0.014*"이노베이션" + 0.013*"미국"')


In [177]:
print(ldamodel.print_topics())

[(0, '0.002*"기술" + 0.002*"배터리" + 0.002*"유출" + 0.002*"화학" + 0.002*"소송" + 0.002*"보안" + 0.002*"산업" + 0.002*"보호" + 0.002*"미국" + 0.002*"핵심"'), (1, '0.055*"기술" + 0.027*"화학" + 0.021*"배터리" + 0.021*"유출" + 0.020*"보안" + 0.018*"소송" + 0.017*"산업" + 0.016*"보호" + 0.014*"이노베이션" + 0.013*"미국"')]


In [178]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
id2word=corpora.Dictionary(data)
id2word.filter_extremes(no_below = 2) #20회 이하로 등장한 단어는 삭제
texts = data
corpus=[id2word.doc2bow(text) for text in texts]


os.environ['MALLET_HOME'] = 'C:/Users/MoTS/topic_modeling/mallet-2.0.8/'

mallet_path = 'C:/Users/MoTS/topic_modeling/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=2, id2word=id2word)